## Structs

We can define a `struct` specifiing the fields and the type they should be.

In [1]:
mutable struct Coly0
    x :: Int32
    y :: Float64
    z :: Bool
    t :: Int32
    u :: Float32
    v :: Bool
    Coly0() = ( K = new())
end;

In [2]:
p = Coly0() 

Coly0(224220232, 0.0, false, 0, 0.0f0, false)

Notice two important details:

- We cannot chose to set one (or several) of the fields of `Coly0`
- The values of the fields when you do   `Coly0()` are set randomly. 

To show that the values of the fields are set ramdonly we can create two different instances of `Coly0`
and see that (probably) they will have different values.

Here julia is just taking a slice of the memory (and whatever bytes are there are used to instanciate the fields)

In [3]:
Coly0() 

Coly0(21, 1.04e-322, false, 1, 3.1f-44, false)

In [4]:
Coly0() 

Coly0(52, 2.5e-322, false, 1, 7.4f-44, false)

## Creating an instance of a struct specifying only a subset of the variables

In Julia a function can have optional arguments. In particular, a constructor for a struct can also have optional  arguments. 

If we want to create a constructor for a struct with the capability to be instanciated only passing one of the values we can do as follows:

In [5]:
mutable struct Coly1
    x :: Int32
    y :: Float64
    z :: Bool
    t :: Int32
    u :: Float32
    v :: Bool
end;

function Coly1(;x=0,y=0,z=false,t=0,u=0,v=false)
    return Coly1(x,y,z,t,u,v)
end

Coly1

Notice that we set some values for the fields of the constructor. In our case all is set to `0` and `false`.

We can create a `Coly1` instance specifying only one of the arguments.

In [6]:
p = Coly1(y=3.14)

Coly1(0, 3.14, false, 0, 0.0f0, false)

Or a subset of the arguments

In [7]:
p2 = Coly1(y=3.14,z=true)

Coly1(0, 3.14, true, 0, 0.0f0, false)

Or none of the arguments


In [8]:
p2 = Coly1()

Coly1(0, 0.0, false, 0, 0.0f0, false)

A question might arise. Once a `Coly1` instance is created, can we know which field was used to instanciate the object?

For example after 

```
p2 = Coly1(y=3.14,z=true)
```

Is called, can we know that we used `y` and `z` to create the instance?

Well in this case we can know because the values passed are different than the default values but it could be  the case that we define
```
p2 = Coly1(y=0,z=true)
```
which would not allow us to know that `y` was set by the user.



In [9]:
mutable struct Coly3
    x :: Union{Int32,Missing}
    y :: Union{Float64, Missing}
    z :: Union{Bool,Missing}
    t :: Union{Int32,Missing}
    u :: Union{Float32,Missing}
    v :: Union{Bool,Missing}
    #coly8() = ( K = new())
end;

function Coly3(;x=missing,y=missing,z=missing,t=missing,u=missing,v=missing)
    return Coly3(x,y,z,t,u,v)
end

Coly3

Now we can create objects with default "missing" to explicitly know that users did not set a value, 
with the convention that argumetns will not be set to missing.

In [10]:
p = Coly3()

Coly3(missing, missing, missing, missing, missing, missing)

In [11]:
p2 = Coly3(y=3)

Coly3(missing, 3.0, missing, missing, missing, missing)

In [12]:
mutable struct Coly4
    x :: Int32
    y :: Float64
    z :: Bool
    t :: Int32
    u :: Float32
    v :: Bool
end;

function Coly4(x;y=0,z=false,t=0,u=0,v=false)
    return Coly4(x,y,z,t,u,v)
end

Coly4

In [13]:
Coly4(4)

Coly4(4, 0.0, false, 0, 0.0f0, false)

## Struct generation with random values in a set of predefined fields

In [14]:
mutable struct Coly_xy
           x :: Int32
           y :: Float64
           z :: Bool
           t :: Int32
           u :: Float32
           v :: Bool
           Coly_xy(x,y) = new(x,y)
       end


In [15]:
Coly_xy(2,3)

Coly_xy(2, 3.0, false, 1, 1.5f-44, false)

In [16]:
Coly_xy(2,4)

Coly_xy(2, 4.0, false, 1, 1.0f-45, false)

## Struct generation with random values in any set of fields

In [33]:
mutable struct coly_rand
    x :: Int32
    y :: Float64
    z :: Bool
    t :: Int32
    u :: Float32
    v :: Bool
    
    function coly_rand(;kwargs...)
        K = new()
        for (key, value) in kwargs
            setfield!(K, key, value)
        end
        return K
    end
    
end

In [88]:
p = coly_rand(y=Float64(3))

coly_rand(8388608, 3.0, false, 1, 2.6329966f-28, true)

Notice though that this is not working

In [90]:
coly_rand(3)

MethodError: MethodError: no method matching coly_rand(::Int64)
Closest candidates are:
  coly_rand(; kwargs...) at In[81]:10

Since we want to use a constructor that is not the defautl we need to specify the conversion of the input types.

In [96]:
field_types = fieldtypes(coly_rand)

(Int32, Float64, Bool, Int32, Float32, Bool)

In [93]:
mutable struct coly_rand
    x :: Int32
    y :: Float64
    z :: Bool
    t :: Int32
    u :: Float32
    v :: Bool
    
    function coly_rand(;kwargs...)
        K = new()
        for (key, value) in kwargs
            field_type_key = typeof(getfield(K,key))
            setfield!(K, key, convert(field_type_key, value))
        end
        return K
    end
    
end

In [94]:
coly_rand(x=2,y=2)

coly_rand(2, 2.0, false, 1, 0.0f0, false)

In [95]:
coly_rand(x=2)

coly_rand(2, 2.2649842505e-314, false, 0, 3.0f-45, false)